<a href="https://colab.research.google.com/github/SisekoC/My-Notebooks/blob/main/Bioinformatics_exercises_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Consider the function get_hits above. Create a variant that allows at most 1 character to
 be different between the sequence and the query words.

In [1]:
def get_hits_with_tolerance(seq, m, w):
    def hamming_distance(s1, s2):
        return sum(1 for x, y in zip(s1, s2) if x != y)

    res = []  # list of tuples
    for i in range(len(seq) - w + 1):
        subseq = seq[i:i + w]
        for query in m:
            if len(query) == w and hamming_distance(subseq, query) <= 1:
                indices = m[query]
                for ind in indices:
                    res.append((ind, i))
    return res


2. a. Write afunction that given two sequences of the same length, determines if they
 have at most d mismatches (d is an argument of the function). The function returns
 True if the number of mismatches is less or equal to d,andFalse otherwise.

In [5]:
def has_at_most_d_mismatches(seq1, seq2, d):
    # Ensure the sequences are of the same length
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be of the same length.")

    # Count the number of mismatches
    mismatch_count = sum(1 for x, y in zip(seq1, seq2) if x != y)

    # Return True if mismatches are less than or equal to d, else False
    return mismatch_count <= d

# Example usage:
seq1 = "ACGTAGC"
seq2 = "ACCTACC"
d = 1
print(has_at_most_d_mismatches(seq1, seq2, d))  # Output: True


False


b. Using the previous function find all approximate matches of a pattern p in a sequence. An approximate match of the pattern can have at most d characters that do
 not match (d is an argument of the function).

In [6]:
def has_at_most_d_mismatches(seq1, seq2, d):
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be of the same length.")
    mismatch_count = sum(1 for x, y in zip(seq1, seq2) if x != y)
    return mismatch_count <= d

def find_approximate_matches(seq, pattern, d):
    w = len(pattern)
    matches = []  # list to store the starting indices of approximate matches
    for i in range(len(seq) - w + 1):
        subseq = seq[i:i + w]
        if has_at_most_d_mismatches(subseq, pattern, d):
            matches.append(i)
    return matches

# Example usage:
seq = "ACGTAGCTAGC"
pattern = "AGC"
d = 1
print(find_approximate_matches(seq, pattern, d))  # Output: [2, 6, 8]


[4, 8]


3. Search in the UniProt database the record for the human protein APAF (O14727). Save it
 in the FASTA format. Using BioPython perform the following operations:
 a. Load the file and check that the protein contains 1248 aminoacids.

In [8]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.5 MB/s eta 0:00:00


In [11]:
from Bio import SeqIO

# Load the FASTA file
fasta_file = "uniprotkb_2025_02_06.fasta"
record = SeqIO.read(fasta_file, "fasta")

# Check the length of the protein
protein_length = len(record.seq)
print(f"The protein contains {protein_length} amino acids.")

# Verify that the protein contains 1248 amino acids
assert protein_length == 1248, "The protein does not contain 1248 amino acids."
print("The protein contains 1248 amino acids.")


The protein contains 1248 amino acids.
The protein contains 1248 amino acids.


b. Using BLASTP, search for sequences with high similarity to this sequence, in the
 “swissprot” database.

I found that the most similar match is the following based on the following criteria: RecName: Full=Apoptotic protease-activating factor 1; Short=APAF-1 [Homo sapiens]

1 It achieved the lowest E-value in the entire list.

2 It had 100% identity percentage, which shows closeness of the two matches.

3 It had the best Query cover, Total Score and Max score. Which shows that the match was of the highest quality you could ever get in the swissport database.

c. Check which the global parameters were used in the search: the database, the substitution matrix, and the gap penalties.

The database is swissport

The substitution matrix is BLOSUM62

The gap penalties are 11,1

 d. List the best alignments returned, showing the accession numbers of the sequences,
 the E value of the alignments, and the alignment length.

The best alignment returned: RecName: Full=Apoptotic protease-activating factor 1; Short=APAF-1 [Homo sapiens], RecName: Full=Apoptotic protease-activating factor 1; Short=APAF-1 [Rattus norvegicus], RecName: Full=Apoptotic protease-activating factor 1; Short=APAF-1 [Mus musculus]

Accession number: O1472.7, Q9EPV5.1, Q88879.3

E values: 0.0, 0.0, 0.0

Alignment length: 1248, 1249, 1249

 e. Repeat the search restricting the target sequences to the organism S. cerevisiae (sug
gestion: use the argument entrez_query in the qblast function).

In [12]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO

# Load the FASTA file
fasta_file = "uniprotkb_2025_02_06.fasta"
record = SeqIO.read(fasta_file, "fasta")

# Perform the BLASTP search
result_handle = NCBIWWW.qblast("blastp", "swissprot", record.seq, entrez_query="Saccharomyces cerevisiae[ORGN]")

# Save the results to a file
blast_result = open("S_cerevisiae_blast_results.xml", "w")
blast_result.write(result_handle.read())
blast_result.close()

# Close the result handle
result_handle.close()


f. Check the results from the last operation, listing the best alignments, and checking
 carefully in each the start position of the alignment in the query and in the sequence.

In [13]:
from Bio.Blast import NCBIXML

# Load the BLAST results file
result_handle = open("S_cerevisiae_blast_results.xml")

# Parse the results
blast_records = NCBIXML.parse(result_handle)
blast_record = next(blast_records)  # Get the first BLAST record


In [14]:
# List to store the best alignments
best_alignments = []

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        best_alignments.append({
            "accession": alignment.accession,
            "e_value": hsp.expect,
            "alignment_length": hsp.align_length,
            "query_start": hsp.query_start,
            "subject_start": hsp.sbjct_start
        })

# Print the best alignments
for alignment in best_alignments:
    print(f"Accession: {alignment['accession']}, E-Value: {alignment['e_value']}, Alignment Length: {alignment['alignment_length']}")
    print(f"Query Start: {alignment['query_start']}, Subject Start: {alignment['subject_start']}\n")


Accession: P25382, E-Value: 8.81845e-25, Alignment Length: 346
Query Start: 629, Subject Start: 158

Accession: P25382, E-Value: 2.72387e-11, Alignment Length: 220
Query Start: 610, Subject Start: 315

Accession: P25382, E-Value: 6.98148e-11, Alignment Length: 331
Query Start: 838, Subject Start: 141

Accession: P25382, E-Value: 1.08419e-10, Alignment Length: 105
Query Start: 1084, Subject Start: 142

Accession: Q05946, E-Value: 6.63484e-18, Alignment Length: 159
Query Start: 655, Subject Start: 489

Accession: Q05946, E-Value: 1.09265e-15, Alignment Length: 218
Query Start: 560, Subject Start: 442

Accession: Q05946, E-Value: 1.33817e-13, Alignment Length: 556
Query Start: 630, Subject Start: 117

Accession: Q05946, E-Value: 9.70153e-08, Alignment Length: 157
Query Start: 623, Subject Start: 69

Accession: Q05946, E-Value: 3.32579e-07, Alignment Length: 174
Query Start: 614, Subject Start: 387

Accession: Q05946, E-Value: 0.00324507, Alignment Length: 75
Query Start: 614, Subject Star

Accession: The unique identifier for the subject sequence in the SwissProt database.

E-Value: The expect value of the alignment, indicating the significance of the match.

Alignment Length: The length of the alignment between the query and the subject sequence.

Query Start: The starting position of the alignment in the query sequence.

Subject Start: The starting position of the alignment in the subject sequence.

g. What do you conclude about the existence of homologous genes in the yeast for the
 human protein APAF ?

Based on the BLASTP search results, it appears that there are homologous genes in *S. cerevisiae* (yeast) for the human protein APAF. The presence of significant alignments with low E-values and substantial alignment lengths suggests that the yeast genome contains sequences that are similar to the human APAF protein.

This indicates that the function of APAF may be conserved across these species, despite the evolutionary distance between them. Homologous genes often retain similar functions, which can be crucial for understanding the role of APAF in yeast and potentially leveraging yeast as a model organism for studying APAF-related processes.